In [63]:
from typing import Union, List
from pathlib import Path

import pandas as pd

import pyomo.environ as pyo
from pyomo.core.base.var import IndexedVar

import src.constants as cst

In [2]:
profiles = pd.DataFrame(
    {
        "Clément": {cst.TECH: cst.LVB, cst.AUT: cst.LVA, cst.EXP: cst.LVD, cst.REL: cst.LVC, cst.SLRY: 467},
        "Maxence": {cst.TECH: cst.LVB, cst.AUT: cst.LVD, cst.EXP: cst.LVD, cst.REL: cst.LVD, cst.SLRY: 321},
        "Josiane": {cst.TECH: cst.LVD, cst.AUT: cst.LVD, cst.EXP: cst.LVA, cst.REL: cst.LVB, cst.SLRY: 373},
        "Jiminy": {cst.TECH: cst.LVD, cst.AUT: cst.LVB, cst.EXP: cst.LVA, cst.REL: cst.LVA, cst.SLRY: 482},
    }
)

profiles

,Clément,Maxence,Josiane,Jiminy
Technicité,B,B,D,D
Autonomie,A,D,D,B
Exposition,D,D,A,A
Relations,C,D,B,A
Salaire,467,321,373,482


In [5]:
weights = pd.DataFrame(
    {
        cst.TECH: {cst.LVA: 50, cst.LVB: 40, cst.LVC: 30, cst.LVD: 20},
        cst.AUT: {cst.LVA: 10, cst.LVB: 8, cst.LVC: 6, cst.LVD: 4},
        cst.EXP: {cst.LVA: 100, cst.LVB: 80, cst.LVC: 60, cst.LVD: 40},
        cst.REL: {cst.LVA: 5, cst.LVB: 4, cst.LVC: 3, cst.LVD: 2},
    }
)

weights

,Technicité,Autonomie,Exposition,Relations
A,50,10,100,5
B,40,8,80,4
C,30,6,60,3
D,20,4,40,2


### Model

#### Set path to ipopt solver

In [44]:
ipopt_path = Path.home().parent.parent / "msys64" / "mingw64" / "bin" / "ipopt.exe"
ipopt_path

WindowsPath('C:/msys64/mingw64/bin/ipopt.exe')

#### Construct Model

In [62]:
def compute_salary(
    *,
    profile: Union[dict, pd.Series],
    weights: Union[IndexedVar, pd.DataFrame],
    skills: List[str],
) -> Union[float, pyo.Expression]:

    index, cols = {}, {}
    if isinstance(weights, pd.DataFrame):
        index, cols = weights.index, weights.columns
        # for indexation below
        weights = weights.stack()
    elif isinstance(weights, IndexedVar):
        index, cols = zip(*list(weights.index_set()))
    else:
        raise ValueError("weights must be a pandas dataframe or a pyomo variable")
    index, cols = set(index), set(cols)

    if set(skills) == cols:
        return sum(weights[(profile[skl], skl)] for skl in skills)
    elif set(skills) == set(index):
        return sum(weights[(skl, profile[skl])] for skl in skills)
    
    raise ValueError(f"Skils ({skills}) don't match weights index/columns")


def extract_gaps(weights_gap: IndexedVar) -> pd.Series:
    
    return pd.Series({idx: var.value for idx, var in weights_gap.items()})


def extract_weights(weights: IndexedVar) -> pd.DataFrame:

    return pd.Series({idx: var.value for idx, var in weights.items()}).unstack()


def solve_model(
    profiles: pd.DataFrame,
    salaries: pd.Series,
    skills: List[str],
    levels: List[str],
):

    model = pyo.ConcreteModel()
    model.weights = pyo.Var(skills, levels, within=pyo.NonNegativeReals)
    model.weights_gap = pyo.Var(skills, within=pyo.NonNegativeReals)

    skill_gaps = {}
    for skl in skills:
        for lv1, lv2 in zip(levels[:-1], levels[1:]):
            expr = model.weights[(skl, lv1)] - model.weights[(skl, lv2)] == model.weights_gap[skl]
            skill_gaps[(skl, lv1, lv2)] = expr

    model.skill_gaps_constraint = pyo.Constraint(skill_gaps.keys(), expr=skill_gaps)

    # Objectif = somme des erreurs au carré
    objexpr = sum(
        (compute_salary(profile=val[skills], weights=model.weights, skills=skills) - salaries[per]) ** 2
        for per, val in profiles.items()
    )

    model.obj = pyo.Objective(expr=objexpr, sense=pyo.minimize)

    solver = pyo.SolverFactory("ipopt", executable=ipopt_path)
    solver.solve(model)

    print(f"Error: {model.obj()}")

    weights_gap = extract_gaps(model.weights_gap)
    weights = extract_gaps(model.weights)

    return pd.concat([weights, weights_gap.rename("Gap")], axis=1)

In [ ]:
model = pyo.ConcreteModel()
model.weights = pyo.Var(cst.SKILLS, cst.LEVELS, within=pyo.NonNegativeReals)
model.weights_gap = pyo.Var(cst.SKILLS, within=pyo.NonNegativeReals)

skill_gaps = {}
for skl in cst.SKILLS:
    for lv1, lv2 in zip(cst.LEVELS[:-1], cst.LEVELS[1:]):
        expr = model.weights[(skl, lv1)] - model.weights[(skl, lv2)] == model.weights_gap[skl]
        skill_gaps[(skl, lv1, lv2)] = expr

model.skill_gaps_constraint = pyo.Constraint(skill_gaps.keys(), expr=skill_gaps)

# Objectif = somme des erreurs au carré
objexpr = sum(
    (compute_salary(profile=val[cst.SKILLS], weights=model.weights) - val[cst.SLRY]) ** 2
    for _, val in profiles.items()
)

model.obj = pyo.Objective(expr=objexpr, sense=pyo.minimize)

solver = pyo.SolverFactory("ipopt", executable=ipopt_path)
solver.solve(model)

model.obj()

1.783912940360237e-16

#### Extract Solutions

In [60]:
gaps_sol = pd.Series(
    {
        idx: var.value for idx, var in model.weights_gap.items()
    }
)
gaps_sol

Technicité    10.897960
Autonomie     37.000000
Exposition     1.265307
Relations     35.000000
dtype: float64

In [61]:
weights_sol = pd.Series(
    {
        idx: var.value for idx, var in model.weights.items()
    }
).unstack()
weights_sol

,A,B,C,D
Autonomie,163.762555,126.762555,89.762555,52.762555
Exposition,105.483944,104.218637,102.953331,101.688024
Relations,174.965871,139.965871,104.965871,69.965871
Technicité,107.481511,96.583551,85.685590,74.787630


### Check

In [57]:
pd.Series(
    {
        per: compute_salary(profile=val[cst.SKILLS], weights=weights_sol)
        for per, val in profiles.items()
    }
)

Clément    467.0
Maxence    321.0
Josiane    373.0
Jiminy     482.0
dtype: float64

In [58]:
profiles

,Clément,Maxence,Josiane,Jiminy
Technicité,B,B,D,D
Autonomie,A,D,D,B
Exposition,D,D,A,A
Relations,C,D,B,A
Salaire,467,321,373,482
